<a href="https://colab.research.google.com/github/caesarcc/pucminas-tcc-fake-news-detection/blob/main/passo02_preparacao_do_PLSUM_sumarizador.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Preparação do Modelo PLSUM, pré-treinado em PT-BR, para sumarização

### Iniciando pela configuração do Google Colab, libs externas e acesso ao Google Drive

In [1]:
# lib para utilizar arquitetura transformers
!pip install -q transformers==4.18.0
# lib necessária para o tokenizador   
!pip install -q sentencepiece
# lib auxiliar para facilitar o scraping de artigos na web
!pip install -q newspaper3k

     |████████████████████████████████| 4.0 MB 5.1 MB/s 
     |████████████████████████████████| 880 kB 49.3 MB/s 
     |████████████████████████████████| 6.6 MB 38.9 MB/s 
     |████████████████████████████████| 596 kB 58.2 MB/s 
     |████████████████████████████████| 86 kB 4.7 MB/s 
     |████████████████████████████████| 1.2 MB 5.2 MB/s 
     |████████████████████████████████| 211 kB 5.1 MB/s 
     |████████████████████████████████| 81 kB 8.6 MB/s 
     |████████████████████████████████| 93 kB 1.6 MB/s 
     |████████████████████████████████| 7.4 MB 59.4 MB/s 


In [ ]:
# Acesso ao Google Driver onde serão salvos os arquivos grandes
# Atenção: Todos arquivos usados também estão disponíveis nas pastas modelos e dados, respectivamente
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
# Importação de bibliotecas
import re
import time
import newspaper
import torch
import pandas as pd
from transformers import T5TokenizerFast
from transformers import T5ForConditionalGeneration

### Carga dos modelos pré-treinados em arquitetura transformers, disponíveis na plataforma [Hugging Face](https://huggingface.co/models)

In [15]:
# Modelo trainado em PT-BR
model_name = 'seidel/plsum-base-ptt5'
# Identifica device gpu ou cpu disponível
device_disponivel = 'cuda' if torch.cuda.is_available() else 'cpu'

# Tokenizador para ptbr
tokenizer = T5TokenizerFast.from_pretrained(model_name)
# Pesos do modelo em PyTorch
model_pt = T5ForConditionalGeneration.from_pretrained(model_name, use_cache=False).to(device_disponivel)

### Coleta automatizada de notícias na capa do portal do UOL

In [ ]:
df_noticias = pd.DataFrame(columns=['titulo', 'texto', 'url'])

uol_paper = newspaper.build('https://noticias.uol.com.br/')
for uol_article in uol_paper.articles:
  try:
    uol_article.download()
    uol_article.parse()
    df_noticias = df_noticias.append({'titulo': uol_article.title, 
                                      'texto': uol_article.text,
                                      'url': uol_article.url}, ignore_index=True)
    time.sleep(5)
  except:
    break

df_noticias.shape

(52, 3)

In [ ]:
# limpa urls e marcadores não textuais
def limpar_textos(texto):
    # Quebra de linha é importante para o modelo de sumarização
    #texto = re.sub(r'(\n|\r)', ' ', texto) 
    # Corrige marcadores não interpretados
    texto = re.sub('(\n|\r)', '', texto)
    # Remove urls no texto
    texto = re.sub(r'(https|http|ftp)?:\/\/(\w|\.|-|\/|\?|\=|\&|\%)*\b', '', texto)
    return texto.strip()

df_noticias["texto"] = df_noticias.texto.apply(limpar_textos)

In [ ]:
# conta palavras
df_noticias['qtde_palavras'] = df_noticias.texto.apply(lambda texto: len(re.findall(r"[\w']+|[.,!?;:/\"]", texto)))
# cria df com as 10 maiores notícias (em quantidade de palavras)
df_noticias_longas = df_noticias.nlargest(10,'qtde_palavras')
df_noticias_longas.head(10)

,titulo,texto,url,qtde_palavras
35,Caso Marielle: 'Nunca havia visto um crime tão...,Primeiro delegado designado para investigar o ...,https://noticias.uol.com.br/ultimas-noticias/a...,2770
29,Vídeo faz alegações enganosas para colocar em ...,Conteúdo investigado: Vídeo em que autor reúne...,https://noticias.uol.com.br/comprova/ultimas-n...,2674
32,"Em novo plano, campanha de Lula muda revogação...",O novo texto com as diretrizes para o programa...,https://www.uol.com.br/eleicoes/2022/06/20/dir...,1513
49,Iphan atropela conselho que cuida de tombament...,São Paulo e BrasíliaO Instituto do Patrimônio ...,https://www1.folha.uol.com.br/ilustrada/2022/0...,1362
33,Herpes zoster: vacina já está disponível em cl...,"Lançada no Brasil neste mês, a vacina contra o...",https://noticias.uol.com.br/ultimas-noticias/a...,1321
36,Congresso tem poder inédito sobre Orçamento e ...,Protagonista da maior renovação política desde...,https://noticias.uol.com.br/ultimas-noticias/a...,1228
46,Absurdos de Bolsonaro com Petrobras são inútei...,Conteúdo exclusivo para assinantesJornalista p...,https://economia.uol.com.br/colunas/jose-paulo...,1146
23,Spirulina: benefícios surpreendentes dessa mic...,"O termo superalimento, usado para designar ali...",https://midiamax.uol.com.br/variedades/saude/2...,1073
0,"Valendo ouro: após reajuste, gasolina é encont...","O reajuste anunciado pela Petrobras de 5,2% na...",https://midiamax.uol.com.br/cotidiano/2022/val...,771
45,"Dólar sobe a R$ 5,186, maior valor em 4 meses;...",O dólar comercial abriu a semana em valorizaçã...,https://economia.uol.com.br/cotacoes/noticias/...,755


### Codifica (Tokeniza) notícias no dataframe

In [ ]:
df_noticias_longas['tokens'] = df_noticias_longas.texto.apply(lambda texto: tokenizer.encode(texto, return_tensors="pt"))
df_noticias_longas['qtde_tokens'] = df_noticias_longas.tokens.apply(lambda tokens: len(tokens[0]))
df_noticias_longas.head(10)

,titulo,texto,url,qtde_palavras,tokens,qtde_tokens
35,Caso Marielle: 'Nunca havia visto um crime tão...,Primeiro delegado designado para investigar o ...,https://noticias.uol.com.br/ultimas-noticias/a...,2770,"[[tensor(3967), tensor(8873), tensor(4086), te...",3223
29,Vídeo faz alegações enganosas para colocar em ...,Conteúdo investigado: Vídeo em que autor reúne...,https://noticias.uol.com.br/comprova/ultimas-n...,2674,"[[tensor(631), tensor(146), tensor(709), tenso...",3112
32,"Em novo plano, campanha de Lula muda revogação...",O novo texto com as diretrizes para o programa...,https://www.uol.com.br/eleicoes/2022/06/20/dir...,1513,"[[tensor(28), tensor(288), tensor(1958), tenso...",1848
49,Iphan atropela conselho que cuida de tombament...,São Paulo e BrasíliaO Instituto do Patrimônio ...,https://www1.folha.uol.com.br/ilustrada/2022/0...,1362,"[[tensor(99), tensor(196), tensor(8), tensor(3...",1570
33,Herpes zoster: vacina já está disponível em cl...,"Lançada no Brasil neste mês, a vacina contra o...",https://noticias.uol.com.br/ultimas-noticias/a...,1321,"[[tensor(5215), tensor(7056), tensor(19), tens...",1678
36,Congresso tem poder inédito sobre Orçamento e ...,Protagonista da maior renovação política desde...,https://noticias.uol.com.br/ultimas-noticias/a...,1228,"[[tensor(743), tensor(5938), tensor(11846), te...",1454
46,Absurdos de Bolsonaro com Petrobras são inútei...,Conteúdo exclusivo para assinantesJornalista p...,https://economia.uol.com.br/colunas/jose-paulo...,1146,"[[tensor(631), tensor(146), tensor(709), tenso...",1355
23,Spirulina: benefícios surpreendentes dessa mic...,"O termo superalimento, usado para designar ali...",https://midiamax.uol.com.br/variedades/saude/2...,1073,"[[tensor(28), tensor(762), tensor(1312), tenso...",1513
0,"Valendo ouro: após reajuste, gasolina é encont...","O reajuste anunciado pela Petrobras de 5,2% na...",https://midiamax.uol.com.br/cotidiano/2022/val...,771,"[[tensor(28), tensor(3044), tensor(12067), ten...",1028
45,"Dólar sobe a R$ 5,186, maior valor em 4 meses;...",O dólar comercial abriu a semana em valorizaçã...,https://economia.uol.com.br/cotacoes/noticias/...,755,"[[tensor(28), tensor(20419), tensor(1618), ten...",884


### Gera resumo dos tokens pelo modelo carregado

In [ ]:
def gerar_resumo(tokens):
  # coloca os tokens no devise
  tensor_tokens = tokens.to(device_disponivel)
  # max_length: O número máximo de tokens a serem gerado.
  # length_penalty: Penalidade exponencial para o comprimento, 1,0 significa sem penalidade.
  # num_beams: Especifica como usar busca heurística (beam search) em vez de busca gulosa (greedy search), 
  #            O modelo tentará manter as 4 hipóteses mais prováveis ​​em cada passo de tempo.
  # early_stopping: A geração seja concluída quando todas as hipóteses (beams) atingirem o final do token de string ( EOS ).
  outputs = model_pt.generate(
      tensor_tokens, 
      max_length=400, 
      length_penalty=2.0, 
      num_beams=4, 
      early_stopping=True)
  return outputs

In [ ]:
# gera os resumos e contar os tokens gerados em cada linha do dataset
df_noticias_longas['tokens_gerados'] = df_noticias_longas.tokens.apply(lambda tokens: gerar_resumo(tokens))
df_noticias_longas['qtde_tokens_gerados'] = df_noticias_longas.tokens_gerados.apply(lambda tokens: len(tokens[0]))
df_noticias_longas.head()

,titulo,texto,url,qtde_palavras,tokens,qtde_tokens,tokens_gerados,qtde_tokens_gerados
35,Caso Marielle: 'Nunca havia visto um crime tão...,Primeiro delegado designado para investigar o ...,https://noticias.uol.com.br/ultimas-noticias/a...,2770,"[[tensor(3967), tensor(8873), tensor(4086), te...",3223,"[[tensor(0, device='cuda:0'), tensor(28, devic...",368
29,Vídeo faz alegações enganosas para colocar em ...,Conteúdo investigado: Vídeo em que autor reúne...,https://noticias.uol.com.br/comprova/ultimas-n...,2674,"[[tensor(631), tensor(146), tensor(709), tenso...",3112,"[[tensor(0, device='cuda:0'), tensor(9, device...",400
32,"Em novo plano, campanha de Lula muda revogação...",O novo texto com as diretrizes para o programa...,https://www.uol.com.br/eleicoes/2022/06/20/dir...,1513,"[[tensor(28), tensor(288), tensor(1958), tenso...",1848,"[[tensor(0, device='cuda:0'), tensor(9, device...",346
49,Iphan atropela conselho que cuida de tombament...,São Paulo e BrasíliaO Instituto do Patrimônio ...,https://www1.folha.uol.com.br/ilustrada/2022/0...,1362,"[[tensor(99), tensor(196), tensor(8), tensor(3...",1570,"[[tensor(0, device='cuda:0'), tensor(9, device...",400
33,Herpes zoster: vacina já está disponível em cl...,"Lançada no Brasil neste mês, a vacina contra o...",https://noticias.uol.com.br/ultimas-noticias/a...,1321,"[[tensor(5215), tensor(7056), tensor(19), tens...",1678,"[[tensor(0, device='cuda:0'), tensor(9, device...",285


### Decodifica tokens de resumo gerados com máximo de 400 palavras

In [ ]:
df_noticias_longas['texto_resumido'] = df_noticias_longas.tokens_gerados.apply(lambda tokens: tokenizer.decode(tokens[0], skip_special_tokens=True))

In [ ]:
# Salva dados gerados para conferencia
df_noticias_longas.head(10)[['titulo','texto','texto_resumido','url']]
df_noticias_longas[['titulo','texto','texto_resumido','url']].to_csv('drive/MyDrive/PUC/TCC/dados/resumos_noticias_teste.csv', sep = ',', index = True)

### Salva modelo no Google Drive para facilitar reuso (evitar novo download)

In [ ]:
CAMINHO_MODELO = "drive/MyDrive/PUC/TCC/modelos/sumarizador_plsum"
model_pt.save_pretrained(f"{CAMINHO_MODELO}")
tokenizer.save_pretrained(f"{CAMINHO_MODELO}")

('drive/MyDrive/PUC/TCC/modelos/sumarizador_plsum/tokenizer_config.json',
 'drive/MyDrive/PUC/TCC/modelos/sumarizador_plsum/special_tokens_map.json',
 'drive/MyDrive/PUC/TCC/modelos/sumarizador_plsum/tokenizer.json')

In [22]:
# Últimos teste com a primeira página do TCC, resumindo para 200 palavras
texto_tcc = """Nos dias de hoje seria ingênuo culpar a popularização da internet pelos problemas da sociedade, pois não há mais como evitar sua expansão, nem tão pouco é possível imaginar o mundo sem a conectividade que conquistamos. No entanto, é importante ressaltar que os indivíduos desta sociedade podem ser perigosamente anônimos e criativos, além de carregados de desejos, histórias e, como não poderia deixar de ser, vieses negativos e positivos.
Tendo em vista tanta criatividade anônima interconectada fica fácil imaginar como surge um boato, uma história repassada sem verificação, ou uma opinião transformada em fato. Estas situações, aparentemente inofensivas no mundo real, tornam-se perigosas no virtual, pois a checagem de fatos não é uma atividade inerente ao ser humano, nem tão pouco há jornalistas suficientes para revisar todo conteúdo que é publicado, a cada segundo, nesta enorme rede.
É justamente neste contexto de geração e consumo de textos, em escala global e ritmo exponencial, que surgiram os sistemas automatizados de checagem e análise de conteúdo textual. A área de estudo deste tipo de sistema é o Processamento de Linguagem Natural (P.L.N.), na qual ainda temos muito para evoluir, visto que somente nos últimos anos o aprendizado profundo se popularizou e foi capaz de surpreender em algumas atividades textuais tipicamente humanas.
Hoje temos redes neurais capazes de criar histórias, resumir conteúdos, responder questionários e até mesmo transferir todo conhecimento adquirido em uma destas tarefas para a realização de outra, assim como nós humanos fazemos. Alguns trabalhos já tiveram grande destaque acadêmico e sua publicação auxilia novos pesquisadores nesta evolução de forma mais assertiva.
"""
tokens = tokenizer.encode(texto_tcc, return_tensors="pt").to(device_disponivel)
gerados = model_pt.generate(
      tokens, 
      max_length=200, 
      length_penalty=2.0, 
      num_beams=4, 
      early_stopping=True)
print(tokenizer.decode(gerados[0], skip_special_tokens=True))


o processamento de linguagem natural (P.L.N.), na qual ainda temos muito para evoluir, visto que somente nos últimos anos o aprendizado profundo se popularizou e foi capaz de surpreender em algumas atividades textuais tipicamente humanas. Hoje temos redes neurais capazes de criar histórias, resumir conteúdos, responder questionários e até mesmo transferir todo conhecimento adquirido em uma destas tarefas para a realização de outra, assim como nós humanos fazemos. Alguns trabalhos já tiveram grande destaque acadêmico e sua publicação auxilia novos pesquisadores nesta evolução de forma mais assertiva. 
